In [1]:
import time
import pyodbc
import pandas as pd
import requests
from bs4 import BeautifulSoup

def update_football_tables():
   
    # EPL STANDINGS
    
    point_table_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
    response = requests.get(point_table_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    standings_table = soup.find('table', {'id': 'results2024-202591_overall'})
    headers = [th.text for th in standings_table.find('thead').find_all('th')]
    rows = standings_table.find('tbody').find_all('tr')
    
    # Prepare the DataFrame
    data = []
    for row in rows:
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    epl_standings = pd.DataFrame(data, columns=headers)
    
    # Filter necessary columns
    epl_standings_sql = epl_standings[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'Pts', 'Last 5', 'Top Team Scorer']]
    
    # Handle NaN values
    epl_standings_sql = epl_standings_sql.where(pd.notnull(epl_standings_sql), None)
    
    # Connect to SQL Server
    server = 'Max\\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    # Upsert logic using MERGE
    for index, row in epl_standings_sql.iterrows():
        merge_query = """
        MERGE EPL_Standings AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source (Rk, Squad, MP, W, D, L, Pts, Last5, TopTeamScorer)
        ON target.Squad = source.Squad
        WHEN MATCHED THEN 
            UPDATE SET 
                Rk = source.Rk, 
                MP = source.MP, 
                W = source.W, 
                D = source.D, 
                L = source.L, 
                Pts = source.Pts, 
                Last5 = source.Last5, 
                TopTeamScorer = source.TopTeamScorer
        WHEN NOT MATCHED THEN 
            INSERT (Rk, Squad, MP, W, D, L, Pts, Last5, TopTeamScorer)
            VALUES (source.Rk, source.Squad, source.MP, source.W, source.D, source.L, source.Pts, source.Last5, source.TopTeamScorer);
        """
        cursor.execute(merge_query, row['Rk'], row['Squad'], row['MP'], row['W'], row['D'], row['L'], row['Pts'], row['Last 5'], row['Top Team Scorer'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest epl standings.")


    # EPL FIXTURES
    
    epl_fixture_url = "https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures"
    
    response = requests.get(epl_fixture_url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    epl_sched_2024_2025 = soup.find('table',{'id': 'sched_2024-2025_9_1'})
    
    headers = [th.text for th in epl_sched_2024_2025.find('thead').find_all('th')]
    
    rows = epl_sched_2024_2025.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in each row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    epl_schd = pd.DataFrame(data, columns=headers)
    
    epl_schd.head()

    epl_schd_sql = epl_schd[['Wk','Day','Date','Time','Home','Score','Away','Venue']]
    # Add a new column 'League' with value 'EPL'
    epl_schd_sql['League'] = 'EPL'
    
    # Handle NaN values
    epl_schd_sql = epl_schd_sql.where(pd.notnull(epl_schd_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in epl_schd_sql.iterrows():
        merge_query = """
        MERGE EPL_Schedule AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?,?)) AS source (Wk, Day, Date, Time, Home, Score, Away,Venue)
        ON target.Wk = source.Wk and target.Date = source.Date and target.Home = source.Home and target.Away = source.Away
        WHEN MATCHED THEN 
            UPDATE SET 
                Wk = source.Wk, 
                Day = source.Day, 
                Date = source.Date, 
                Time = source.Time, 
                Home = source.Home, 
                Score = source.Score, 
                Away = source.Away,
                Venue = source.Venue;
       
        """
        cursor.execute(merge_query, row['Wk'], row['Day'], row['Date'], row['Time'], row['Home'], row['Score'], row['Away'],row['Venue'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest epl schd.")


    # EPL TOP SCORERES
    
    epl_scores_url = "https://fbref.com/en/stathead/player_comparison.cgi?request=1&sum=0&comp_type=spec&dom_lg=1&spec_comps=9&player_id1=e342ad68&p1yrfrom=2024-2025&player_id2=1f44ac21&p2yrfrom=2024-2025&player_id3=8e92be30&p3yrfrom=2024-2025&player_id4=dc7f8a28&p4yrfrom=2024-2025&player_id5=4e9a0555&p5yrfrom=2024-2025&player_id6=6afaebf2&p6yrfrom=2024-2025&player_id7=dc62b55d&p7yrfrom=2024-2025"
    
    # Fetch the page content
    response = requests.get(epl_scores_url)
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the scorers table using its ID
    scorers = soup.find('table', {'id': 'standard_stats'})
    
    # Extract only the last row of headers (normal header row)
    headers = [th.text.strip() for th in scorers.find('thead').find_all('tr')[-1].find_all('th')]
    
    # Extract table rows
    rows = scorers.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in the row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    # Create a DataFrame
    epl_scorers = pd.DataFrame(data, columns=headers)
    
    
    # Get the column names as a list
    columns = list(epl_scorers.columns)
    
    # Replace only the first occurrence of 'Gls' with 'Goals'
    for i, col in enumerate(columns):
        if col == 'Gls':
            columns[i] = 'Goals'
            break  
    
    # Assign the updated column names back to the DataFrame
    epl_scorers.columns = columns
    
    # Preview the DataFrame
    epl_scorers.head()
    
    epl_scorers_sql = epl_scorers[['Player','Span','Nation','Pos','Squad','MP','Goals']]
    
    epl_scorers_sql['league']='Epl'
    
    epl_scorers_sql.head()
    

    # Handle NaN values
    epl_scorers_sql = epl_scorers_sql.where(pd.notnull(epl_scorers_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    # Clear the existing table
    try:
        truncate_query = "TRUNCATE TABLE EPL_TopScorers"  # Use TRUNCATE for faster clearing if no foreign key constraints
        cursor.execute(truncate_query)
        print("Table cleared successfully.")
    except Exception as e:
        print(f"Error clearing the table: {e}")
    
    # Insert new data
    try:
        insert_query = """
        INSERT INTO  EPL_TopScorers(Player,Span,Nation,Pos,Squad,MP,Goals,league)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """
        for index, row in epl_scorers_sql.iterrows():
            cursor.execute(insert_query, row['Player'], row['Span'], row['Nation'], row['Pos'], row['Squad'], row['MP'], row['Goals'], row['league'])
    
        # Commit the transaction
        conn.commit()
        print("Table updated successfully with the latest epl Top Scoreres.")
    except Exception as e:
        print(f"Error inserting new data: {e}")
    
    # Close the connection
    cursor.close()
    conn.close()


# Lalig stats 

    
    # LALIGA STANDINGS
    
    laliga_url = "https://fbref.com/en/comps/12/La-Liga-Stats"
    
    # Fetch the data
    response = requests.get(laliga_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the standings table
    standings_table = soup.select('table.stats_table')[0]
    
    # Extract headers
    headers = [th.text for th in standings_table.select('thead th')]
    
    # Extract rows
    rows = standings_table.select('tbody tr')
    data = []
    for row in rows:
        row_data = [td.text.strip() for td in row.select('td')]
        # Include rank from <th> (not inside <td>)
        rank = row.select_one('th').text.strip()
        data.append([rank] + row_data)
    
    # Create a DataFrame
    laliga_standings = pd.DataFrame(data, columns=headers)
    
    laliga_standings.head()
    
    
    # Filter necessary columns
    laliga_standings_sql = laliga_standings[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'Pts', 'Last 5', 'Top Team Scorer']]
    
    laliga_standings_sql['league'] = 'Laliga'
    
    laliga_standings_sql.rename(
        columns= {
            'Last 5': 'Last_5',
            'Top Team Scorer' : 'Top_Team_Scorer'
        },inplace = True
    )
    
    laliga_standings_sql.head()

    
    # Handle NaN values
    laliga_standings_sql = laliga_standings_sql.where(pd.notnull(laliga_standings_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in laliga_standings_sql.iterrows():
        merge_query = """
        MERGE laliga_Standings AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source (Rk, Squad, MP, W, D, L, Pts, Last_5, Top_Team_Scorer)
        ON target.Squad = source.Squad
        WHEN MATCHED THEN 
            UPDATE SET 
                Rk = source.Rk, 
                MP = source.MP, 
                W = source.W, 
                D = source.D, 
                L = source.L, 
                Pts = source.Pts, 
                Last_5 = source.Last_5, 
                Top_Team_Scorer = source.Top_Team_Scorer
        WHEN NOT MATCHED THEN 
            INSERT (Rk, Squad, MP, W, D, L, Pts, Last_5, Top_Team_Scorer)
            VALUES (source.Rk, source.Squad, source.MP, source.W, source.D, source.L, source.Pts, source.Last_5, source.Top_Team_Scorer);
        """
        cursor.execute(merge_query, row['Rk'], row['Squad'], row['MP'], row['W'], row['D'], row['L'], row['Pts'], row['Last_5'], row['Top_Team_Scorer'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest laliga standings.")


# LALIGA FIXTURES
    
    lal_fixture_url = "https://fbref.com/en/comps/12/schedule/La-Liga-Scores-and-Fixtures"
    
    response = requests.get(lal_fixture_url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    lal_sched_2024_2025 = soup.find('table',{'id': 'sched_2024-2025_12_1'})
    
    headers = [th.text for th in lal_sched_2024_2025.find('thead').find_all('th')]
    
    rows = lal_sched_2024_2025.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in each row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    lal_schd = pd.DataFrame(data, columns=headers)
    
    lal_schd.head()
    
    lal_schd_sql = lal_schd[['Wk','Day','Date','Time','Home','Score','Away','Venue']]
    
    lal_schd_sql['league'] = 'Laliga'
    
    lal_schd_sql.head()

       
    # Handle NaN values
    lal_schd_sql = lal_schd_sql.where(pd.notnull(lal_schd_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in lal_schd_sql.iterrows():
        merge_query = """
        MERGE LaLiga_Schedule AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?,?)) AS source (Wk, Day, Date, Time, Home, Score, Away,Venue)
        ON target.Wk = source.Wk and target.Date = source.Date and target.Home = source.Home and target.Away = source.Away
        WHEN MATCHED THEN 
            UPDATE SET 
                Wk = source.Wk, 
                Day = source.Day, 
                Date = source.Date, 
                Time = source.Time, 
                Home = source.Home, 
                Score = source.Score, 
                Away = source.Away,
                Venue = source.Venue;
       
        """
        cursor.execute(merge_query, row['Wk'], row['Day'], row['Date'], row['Time'], row['Home'], row['Score'], row['Away'],row['Venue'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest laliga schd.")


    # LALIGA SCORERS
    
    lal_scores_url = "https://fbref.com/en/stathead/player_comparison.cgi?request=1&sum=0&comp_type=spec&dom_lg=1&spec_comps=12&player_id1=8d78e732&p1yrfrom=2024-2025&player_id2=3423f250&p2yrfrom=2024-2025&player_id3=42fd9c7f&p3yrfrom=2024-2025&player_id4=8f3565b3&p4yrfrom=2024-2025&player_id5=0c61c77c&p5yrfrom=2024-2025&player_id6=7111d552&p6yrfrom=2024-2025&player_id7=819aa8e7&p7yrfrom=2024-2025"
    
    # Fetch the page content
    response = requests.get(lal_scores_url)
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the scorers table using its ID
    scorers = soup.find('table', {'id': 'standard_stats'})
    
    # Extract only the last row of headers (normal header row)
    headers = [th.text.strip() for th in scorers.find('thead').find_all('tr')[-1].find_all('th')]
    
    # Extract table rows
    rows = scorers.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in the row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    # Create a DataFrame
    lal_scorers = pd.DataFrame(data, columns=headers)
    
    
    columns = list(lal_scorers.columns)
    
    # Replace only the first occurrence of 'Gls' with 'Goals'
    for i, col in enumerate(columns):
        if col == 'Gls':
            columns[i] = 'Goals'
            break  # Exit the loop after replacing the first occurrence
    
    # Assign the updated column names back to the DataFrame
    lal_scorers.columns = columns
    
    lal_scorers_sql = lal_scorers[['Player','Span','Nation','Pos','Squad','MP','Goals']]
    
    lal_scorers_sql['league']='Laliga'
    
    lal_scorers_sql.head()
    

    # Handle NaN values
    lal_scorers_sql = lal_scorers_sql.where(pd.notnull(lal_scorers_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    # Clear the existing table
    try:
        truncate_query = "TRUNCATE TABLE laliga_TopScorers"  # Use TRUNCATE for faster clearing if no foreign key constraints
        cursor.execute(truncate_query)
        print("Table cleared successfully.")
    except Exception as e:
        print(f"Error clearing the table: {e}")
    
    # Insert new data
    try:
        insert_query = """
        INSERT INTO  laliga_TopScorers (Player,Span,Nation,Pos,Squad,MP,Goals,league)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """
        for index, row in lal_scorers_sql.iterrows():
            cursor.execute(insert_query, row['Player'], row['Span'], row['Nation'], row['Pos'], row['Squad'], row['MP'], row['Goals'], row['league'])
    
        # Commit the transaction
        conn.commit()
        print("Table updated successfully with the latest laliga top scorers.")
    except Exception as e:
        print(f"Error inserting new data: {e}")
    
    # Close the connection
    cursor.close()
    conn.close()

#BUNDESLIGA
    
    # BUNDESLIGA STANDING
    
    bundesliga_url = "https://fbref.com/en/comps/20/Bundesliga-Stats"
    
    # Fetch the data
    response = requests.get(bundesliga_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the standings table
    standings_table = soup.select('table.stats_table')[0]
    
    # Extract headers
    headers = [th.text for th in standings_table.select('thead th')]
    
    # Extract rows
    
    rows = standings_table.select('tbody tr')
    data = []
    for row in rows:
        row_data = [td.text.strip() for td in row.select('td')]
        # Include rank from <th> (not inside <td>)
        rank = row.select_one('th').text.strip()
        data.append([rank] + row_data)
    
    
    # Create a DataFrame
    bundesliga_standings = pd.DataFrame(data, columns=headers)
    
    bundesliga_standings.head()
    
    
    bundesliga_standings_sql = bundesliga_standings[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'Pts', 'Last 5', 'Top Team Scorer']]
    
    bundesliga_standings['league'] = 'Laliga'
    
    bundesliga_standings_sql.rename(
        columns= {
            'Last 5': 'Last_5',
            'Top Team Scorer' : 'Top_Team_Scorer'
        },inplace = True
    )
    
    bundesliga_standings_sql.head()


    # Handle NaN values
    bundesliga_standings_sql = bundesliga_standings_sql.where(pd.notnull(bundesliga_standings_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in bundesliga_standings_sql.iterrows():
        merge_query = """
        MERGE bun_Standings AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source (Rk, Squad, MP, W, D, L, Pts, Last_5, Top_Team_Scorer)
        ON target.Squad = source.Squad
        WHEN MATCHED THEN 
            UPDATE SET 
                Rk = source.Rk, 
                MP = source.MP, 
                W = source.W, 
                D = source.D, 
                L = source.L, 
                Pts = source.Pts, 
                Last_5 = source.Last_5, 
                Top_Team_Scorer = source.Top_Team_Scorer
        WHEN NOT MATCHED THEN 
            INSERT (Rk, Squad, MP, W, D, L, Pts, Last_5, Top_Team_Scorer)
            VALUES (source.Rk, source.Squad, source.MP, source.W, source.D, source.L, source.Pts, source.Last_5, source.Top_Team_Scorer);
        """
        cursor.execute(merge_query, row['Rk'], row['Squad'], row['MP'], row['W'], row['D'], row['L'], row['Pts'], row['Last_5'], row['Top_Team_Scorer'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest bundesliga standings.")



     
    # BUNDESLIGA FIXTURE
    
    Bundesliga_fixture_url = "https://fbref.com/en/comps/20/schedule/Bundesliga-Scores-and-Fixtures"
    
    response = requests.get(Bundesliga_fixture_url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    bun_sched_2024_2025 = soup.find('table',{'id': 'sched_2024-2025_20_1'})
    
    headers = [th.text for th in bun_sched_2024_2025.find('thead').find_all('th')]
    
    rows = bun_sched_2024_2025.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in each row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    bun_schd = pd.DataFrame(data, columns=headers)
    
    bun_schd.head() 
    
    bun_schd_sql = bun_schd[['Wk','Day','Date','Time','Home','Score','Away','Venue']]
    
    bun_schd_sql['league'] = 'bundesliga'
    
    bun_schd_sql.head()

   
    # Handle NaN values
    bun_schd_sql = bun_schd_sql.where(pd.notnull(bun_schd_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in bun_schd_sql.iterrows():
        merge_query = """
        MERGE Bundesliga_Schedule AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?,?)) AS source (Wk, Day, Date, Time, Home, Score, Away,Venue)
        ON target.Wk = source.Wk and target.Date = source.Date and target.Home = source.Home and target.Away = source.Away
        WHEN MATCHED THEN 
            UPDATE SET 
                Wk = source.Wk, 
                Day = source.Day, 
                Date = source.Date, 
                Time = source.Time, 
                Home = source.Home, 
                Score = source.Score, 
                Away = source.Away,
                Venue = source.Venue;
       
        """
        cursor.execute(merge_query, row['Wk'], row['Day'], row['Date'], row['Time'], row['Home'], row['Score'], row['Away'],row['Venue'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest bundesliga schd.")

    
    # BUNDESLIGA SCORERES
    
    bun_scores_url = "https://fbref.com/en/stathead/player_comparison.cgi?request=1&sum=0&comp_type=spec&dom_lg=1&spec_comps=20&player_id1=21a66f6a&p1yrfrom=2024-2025&player_id2=0e0102eb&p2yrfrom=2024-2025&player_id3=05f99a4a&p3yrfrom=2024-2025&player_id4=258a6f4d&p4yrfrom=2024-2025&player_id5=2c0558b8&p5yrfrom=2024-2025&player_id6=5d4f7d61&p6yrfrom=2024-2025&player_id7=1fffae99&p7yrfrom=2024-2025"
    
    # Fetch the page content
    response = requests.get(bun_scores_url)
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the scorers table using its ID
    scorers = soup.find('table', {'id': 'standard_stats'})
    
    # Extract only the last row of headers (normal header row)
    headers = [th.text.strip() for th in scorers.find('thead').find_all('tr')[-1].find_all('th')]
    
    # Extract table rows
    rows = scorers.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in the row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    # Create a DataFrame
    bun_scorers = pd.DataFrame(data, columns=headers)
    
    columns = list(bun_scorers.columns)
    
    # Replace only the first occurrence of 'Gls' with 'Goals'
    for i, col in enumerate(columns):
        if col == 'Gls':
            columns[i] = 'Goals'
            break  # Exit the loop after replacing the first occurrence
    
    # Assign the updated column names back to the DataFrame
    bun_scorers.columns = columns
    # Preview the DataFrame
    bun_scorers.head()
    
  
    # Assign the updated column names back to the DataFrame
    bun_scorers.columns = columns
    
    bun_scorers_sql = bun_scorers[['Player','Span','Nation','Pos','Squad','MP','Goals']]
    
    bun_scorers_sql['league']='bundesliga'
    
    bun_scorers_sql.head()

    # Handle NaN values
    bun_scorers_sql = bun_scorers_sql.where(pd.notnull(bun_scorers_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    # Clear the existing table
    try:
        truncate_query = "TRUNCATE TABLE bundesliga_TopScorers"  # Use TRUNCATE for faster clearing if no foreign key constraints
        cursor.execute(truncate_query)
        print("Table cleared successfully.")
    except Exception as e:
        print(f"Error clearing the table: {e}")
    
    # Insert new data
    try:
        insert_query = """
        INSERT INTO  bundesliga_TopScorers (Player,Span,Nation,Pos,Squad,MP,Goals,league)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """
        for index, row in bun_scorers_sql.iterrows():
            cursor.execute(insert_query, row['Player'], row['Span'], row['Nation'], row['Pos'], row['Squad'], row['MP'], row['Goals'], row['league'])
    
        # Commit the transaction
        conn.commit()
        print("Table updated successfully with the latest bundesliga top scorers.")
    except Exception as e:
        print(f"Error inserting new data: {e}")


# #SERIA STATS
    
    # SERIA STANDINGS
    
    serie_a_url = "https://fbref.com/en/comps/11/Serie-A-Stats"
    
    # Fetch the data
    response = requests.get(serie_a_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the standings table
    standings_table = soup.select('table.stats_table')[0]
    
    # Extract headers
    headers = [th.text for th in standings_table.select('thead th')]
    
    # Extract rows
    rows = standings_table.select('tbody tr')
    data = []
    for row in rows:
        row_data = [td.text.strip() for td in row.select('td')]
        # Include rank from <th> (not inside <td>)
        rank = row.select_one('th').text.strip()
        data.append([rank] + row_data)
    
    # Create a DataFrame
    seria_standings = pd.DataFrame(data, columns=headers)
    
    
    seria_standings.head()
    
    seria_standings_sql = seria_standings[['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'Pts', 'Last 5', 'Top Team Scorer']]
    
    seria_standings_sql['league'] = 'Laliga'
    
    seria_standings_sql.rename(
        columns= {
            'Last 5': 'Last_5',
            'Top Team Scorer' : 'Top_Team_Scorer'
        },inplace = True
    )
    
    seria_standings_sql.head()

    
    # Handle NaN values
    seria_standings_sql = seria_standings_sql.where(pd.notnull(seria_standings_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in seria_standings_sql.iterrows():
        merge_query = """
        MERGE seria_Standings AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)) AS source (Rk, Squad, MP, W, D, L, Pts, Last_5, Top_Team_Scorer)
        ON target.Squad = source.Squad
        WHEN MATCHED THEN 
            UPDATE SET 
                Rk = source.Rk, 
                MP = source.MP, 
                W = source.W, 
                D = source.D, 
                L = source.L, 
                Pts = source.Pts, 
                Last_5 = source.Last_5, 
                Top_Team_Scorer = source.Top_Team_Scorer
        WHEN NOT MATCHED THEN 
            INSERT (Rk, Squad, MP, W, D, L, Pts, Last_5, Top_Team_Scorer)
            VALUES (source.Rk, source.Squad, source.MP, source.W, source.D, source.L, source.Pts, source.Last_5, source.Top_Team_Scorer);
        """
        cursor.execute(merge_query, row['Rk'], row['Squad'], row['MP'], row['W'], row['D'], row['L'], row['Pts'], row['Last_5'], row['Top_Team_Scorer'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest seria standings.")



    # SERIA FIXTURE
    
    seria_fixture_url = "https://fbref.com/en/comps/11/schedule/Serie-A-Scores-and-Fixtures"
    
    response = requests.get(seria_fixture_url)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    seria_sched_2024_2025 = soup.find('table',{'id': 'sched_2024-2025_11_1'})
    
    headers = [th.text for th in seria_sched_2024_2025.find('thead').find_all('th')]
    
    rows = seria_sched_2024_2025.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in each row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    seria_schd = pd.DataFrame(data, columns=headers)
    
    seria_schd.head()
    
    seria_schd_sql = seria_schd[['Wk','Day','Date','Time','Home','Score','Away','Venue']]
    
    seria_schd_sql['league'] = 'Seria'
    
   


    seria_schd_sql = seria_schd_sql.where(pd.notnull(seria_schd_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    
    # Upsert logic using MERGE
    for index, row in seria_schd_sql.iterrows():
        merge_query = """
        MERGE Seria_Schedule AS target
        USING (VALUES (?, ?, ?, ?, ?, ?, ?,?)) AS source (Wk, Day, Date, Time, Home, Score, Away,Venue)
        ON target.Wk = source.Wk and target.Date = source.Date and target.Home = source.Home and target.Away = source.Away
        WHEN MATCHED THEN 
            UPDATE SET 
                Wk = source.Wk, 
                Day = source.Day, 
                Date = source.Date, 
                Time = source.Time, 
                Home = source.Home, 
                Score = source.Score, 
                Away = source.Away,
                Venue = source.Venue;
       
        """
        cursor.execute(merge_query, row['Wk'], row['Day'], row['Date'], row['Time'], row['Home'], row['Score'], row['Away'],row['Venue'])
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()
    
    print("Table updated successfully with the latest seria schd.")

 
    # SERIA SCORERS
    
    seria_scores_url = "https://fbref.com/en/stathead/player_comparison.cgi?request=1&sum=0&comp_type=spec&dom_lg=1&spec_comps=11&player_id1=6f8cd6d0&p1yrfrom=2024-2025&player_id2=e7695e6c&p2yrfrom=2024-2025&player_id3=3d50bcdb&p3yrfrom=2024-2025&player_id4=7c104bb7&p4yrfrom=2024-2025&player_id5=79443529&p5yrfrom=2024-2025&player_id6=83c06f3a&p6yrfrom=2024-2025&player_id7=da76bab4&p7yrfrom=2024-2025"
    
    # Fetch the page content
    response = requests.get(seria_scores_url)
    
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Locate the scorers table using its ID
    scorers = soup.find('table', {'id': 'standard_stats'})
    
    # Extract only the last row of headers (normal header row)
    headers = [th.text.strip() for th in scorers.find('thead').find_all('tr')[-1].find_all('th')]
    
    # Extract table rows
    rows = scorers.find('tbody').find_all('tr')
    data = []
    for row in rows:
        # Extract all cells in the row
        cells = row.find_all(['td', 'th'])
        data.append([cell.text.strip() for cell in cells])
    
    # Create a DataFrame
    seria_scorers = pd.DataFrame(data, columns=headers)
    
    columns = list(seria_scorers.columns)
    
    # Replace only the first occurrence of 'Gls' with 'Goals'
    for i, col in enumerate(columns):
        if col == 'Gls':
            columns[i] = 'Goals'
            break  # Exit the loop after replacing the first occurrence
    
    # Assign the updated column names back to the DataFrame
    seria_scorers.columns = columns
    
    # Preview the DataFrame
    seria_scorers.head()
    
    # Assign the updated column names back to the DataFrame
    seria_scorers.columns = columns
    
    seria_scorers_sql = seria_scorers[['Player','Span','Nation','Pos','Squad','MP','Goals']]
    
    seria_scorers_sql['league']='seria'
    
    seria_scorers_sql.head()


    # Handle NaN values
    seria_scorers_sql = seria_scorers_sql.where(pd.notnull(seria_scorers_sql), None)
    
    # Connect to SQL Server
    server = 'Max\SQLEXPRESS01'
    database = 'Football'
    conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;")
    cursor = conn.cursor()
    
    
    # Clear the existing table
    try:
        truncate_query = "TRUNCATE TABLE seria_TopScorers"  # Use TRUNCATE for faster clearing if no foreign key constraints
        cursor.execute(truncate_query)
        print("Table cleared successfully.")
    except Exception as e:
        print(f"Error clearing the table: {e}")
    
    # Insert new data
    try:
        insert_query = """
        INSERT INTO  seria_TopScorers (Player,Span,Nation,Pos,Squad,MP,Goals,league)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """
        for index, row in seria_scorers_sql.iterrows():
            cursor.execute(insert_query, row['Player'], row['Span'], row['Nation'], row['Pos'], row['Squad'], row['MP'], row['Goals'], row['league'])
    
        # Commit the transaction
        conn.commit()
        print("Table updated successfully with the latest seria Top Scorers.")
    except Exception as e:
        print(f"Error inserting new data: {e}")
    
    # Close the connection
    cursor.close()
    conn.close()


while(True):
    update_football_tables()
    time.sleep(60)




<>:99: SyntaxWarning: invalid escape sequence '\S'
<>:188: SyntaxWarning: invalid escape sequence '\S'
<>:273: SyntaxWarning: invalid escape sequence '\S'
<>:344: SyntaxWarning: invalid escape sequence '\S'
<>:429: SyntaxWarning: invalid escape sequence '\S'
<>:513: SyntaxWarning: invalid escape sequence '\S'
<>:586: SyntaxWarning: invalid escape sequence '\S'
<>:675: SyntaxWarning: invalid escape sequence '\S'
<>:754: SyntaxWarning: invalid escape sequence '\S'
<>:825: SyntaxWarning: invalid escape sequence '\S'
<>:915: SyntaxWarning: invalid escape sequence '\S'
<>:99: SyntaxWarning: invalid escape sequence '\S'
<>:188: SyntaxWarning: invalid escape sequence '\S'
<>:273: SyntaxWarning: invalid escape sequence '\S'
<>:344: SyntaxWarning: invalid escape sequence '\S'
<>:429: SyntaxWarning: invalid escape sequence '\S'
<>:513: SyntaxWarning: invalid escape sequence '\S'
<>:586: SyntaxWarning: invalid escape sequence '\S'
<>:675: SyntaxWarning: invalid escape sequence '\S'
<>:754: Syntax

Table updated successfully with the latest epl standings.


C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epl_schd_sql['League'] = 'EPL'


Table updated successfully with the latest epl schd.


C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epl_scorers_sql['league']='Epl'


...

Table updated successfully with the latest seria schd.


C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seria_scorers_sql['league']='seria'


Table cleared successfully.
Table updated successfully with the latest seria Top Scorers.


C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:99: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\SQLEXPRESS01'
C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:188: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\SQLEXPRESS01'
C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:273: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\SQLEXPRESS01'
C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:344: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\SQLEXPRESS01'
C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:429: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\SQLEXPRESS01'
C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:513: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\SQLEXPRESS01'
C:\Users\Rohith R\AppData\Local\Temp\ipykernel_8328\1104457689.py:586: SyntaxWarning: invalid escape sequence '\S'
  server = 'Max\

KeyboardInterrupt: 